In [3]:

import subprocess

with open('requirements.txt', 'r') as file:
    packages = file.read().splitlines()
for package in packages:
    subprocess.check_call(['pip', 'install', package])

conn = snowflake.connector.connect(
    user='',
    password='',
    account='',  # Correct format for the account parameter
    warehouse='',
    database='',
    schema='',
    role=''
)
openai.api_key = ''

snowflake_table = ""
snowflake_shortname = ""

###########################################
# SCRIPT NEEDS TO RUN daily
#############################################

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "surremor-ocf-69d6a4754b0a.json"

# ACCESS INFORMATION
###########################################
access_key_id =''
secret_access_key=''

batch_size = 100000
company = ''
###########################################
#### Bucket Infro
#############################################
bucket = ''
folder = ''
#############################################
s3 = boto3.client('s3',
                  aws_access_key_id=access_key_id,
                  aws_secret_access_key=secret_access_key,
                  region_name='us-east-1')


current_date = datetime.now().strftime('%Y-%m-%d')


api_token = 'c0b6694bb13c9f6bb194265530e48041'
url = f"https://api.serpstat.com/v4/?token={api_token}"


staging_table = "CONTENT.STAGING_TABLES.PRODUCT_CONTENTS"
batch_size = 100000
snowflake_table = "CONTENT.BLOG_POSTS.PRODUCT_CONTENTS"
company_id= 1

try:
    cur = conn.cursor()
    query = """ 
        SELECT DISTINCT 
            Q.CATEGORY, 
            COALESCE(RKW2.KEYWORD, RKW.KEYWORD) AS GET_RELATED_KEYWORD, 
            COALESCE(RW.KEYWORD, RW1.KEYWORD) AS GET_KEYWORDS, 
            COALESCE(GS.KEYWORD, GS1.KEYWORD) AS SUGGESTED_KEYWORD, 
            FROM 
            GOOGLE_BIGQUERY.SEARCH_CONSOLE_RAW.QUERY_CATEGORY Q
        LEFT JOIN 
            SERPSTAT.KEYWORD_RAW_DATA.GETRELATEDKEYWORDS RKW 
            ON RKW.KEYWORD_SEARCHED = Q.CATEGORY
        LEFT JOIN 
            SERPSTAT.KEYWORD_RAW_DATA.GETRELATEDKEYWORDS RKW2 
            ON RKW2.KEYWORD_SEARCHED = Q.QUERY
        LEFT JOIN 
            SERPSTAT.KEYWORD_RAW_DATA.GETKEYWORDS RW 
            ON Q.QUERY = RW.KEYWORD_SEARCHED
        LEFT JOIN 
            SERPSTAT.KEYWORD_RAW_DATA.GETKEYWORDS RW1 
            ON Q.CATEGORY = RW1.KEYWORD_SEARCHED

        LEFT JOIN 
            SERPSTAT.KEYWORD_RAW_DATA.GETSUGGESTIONS GS 
            ON Q.QUERY = GS.KEYWORD_SEARCHED
        LEFT JOIN 
            SERPSTAT.KEYWORD_RAW_DATA.GETSUGGESTIONS GS1 
            ON Q.CATEGORY = GS.KEYWORD_SEARCHED    
        WHERE 
            COALESCE(GS.KEYWORD, GS1.KEYWORD) IS NOT NULL AND
            COALESCE(RW.KEYWORD, RW1.KEYWORD) IS NOT NULL AND
            COALESCE(GS.KEYWORD, GS1.KEYWORD) IS NOT NULL ;

            """  
    cur.execute(query)
    results = cur.fetchall()
    column_names = [desc[0] for desc in cur.description]
    df = pd.DataFrame(results, columns=column_names)

finally:
    print("failed")

def generate_text(prompt, max_tokens=150):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # You can use "gpt-4" if available on your account
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        n=1,
        stop=None,
        temperature=0.7  # Controls randomness (higher = more random)
    )
    return response['choices'][0]['message']['content'].strip()
def generate_blog_post(product_name, product_description, suggested_keywords, query_keywords):
    prompt = (f"Write a detailed blog post about {product_name}. "
              f"Description: {product_description}. Highlight the following: {suggested_keywords}, "
              f"and target the following search queries: {query_keywords}. "
              "Mention that readers can also check out our social media post for more updates.")
    return generate_text(prompt, max_tokens=400)

def generate_social_post(product_name, product_description, suggested_keywords, query_keywords):
    hashtags = f"#{suggested_keywords.replace(' ', '')} #{query_keywords.replace(' ', '')}"
    
    prompt = (f"Write a short social media post about {product_name}. "
              f"Description: {product_description}. Focus on {suggested_keywords} and use the keywords "
              f"{query_keywords}. Include a call to action telling users to check out our blog post for more details. "
              "Also, generate a caption and relevant hashtags.")
    
    social_post = generate_text(prompt, max_tokens=100)
    
    # Append hashtags and call to action for blog post
    full_social_post = f"{social_post}\n\nRead more in our blog post! [Insert Blog Link Here]\n{hashtags}"
    return full_social_post

final_df = pd.DataFrame()

counter = 0

for index, row in df.iterrows():
    if counter >= 10:
        break  

    # Use your specified variables
    category = row[0]          # 'Category'
    related = row[1]           # 'Related'
    other_keywords = row[2]    # 'Other Keywords'
    suggested = row[3]         # 'Suggested'

    # Prepare the input for the post generation
    product_name = category
    product_description = related
    query_keywords = other_keywords  # Single value
    suggested_keywords = suggested   # Single value

    # Generate blog and social media posts
    blog_post = generate_blog_post(product_name, product_description, suggested_keywords, query_keywords)
    social_post = generate_social_post(product_name, product_description, suggested_keywords, query_keywords)

    # Create a DataFrame with the generated content
    result_data = {
        'Product Name': [product_name],
        'Product Description': [product_description],
        'Suggested Keywords': [suggested_keywords],
        'Query Keywords': [query_keywords],
        'Blog Post': [blog_post],
        'Social Media Post': [social_post]
    }

    df_generated = pd.DataFrame(result_data)

    # Concatenate the newly generated DataFrame to the final DataFrame
    final_df = pd.concat([final_df, df_generated], ignore_index=True)

    # Increment the counter
    counter += 1
print(final_df)
final_df.to_csv('generated_content_with_description.csv', index=False)


final_df['company_id'] = company_id
final_df['DATE_INSERTED'] = current_date
final_df['company'] = company
final_df
# Batch processing
for i in range(len(final_df) // batch_size + 1):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(final_df))
    batch_data = final_df[start_idx:end_idx]
    file_name = f'batch_{i+1}.csv'
    batch_data.to_csv(file_name, index=False)
    s3.upload_file(file_name, bucket, f'{folder}/{file_name}')
    print(f"Uploaded {file_name} to S3.")

copy_command = f"""
COPY INTO {staging_table}
FROM 's3://{bucket}/{folder}/{file_name}'
CREDENTIALS = (AWS_KEY_ID='{access_key_id}' AWS_SECRET_KEY='{secret_access_key}')
FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER=1)
ON_ERROR='CONTINUE';
"""

merge_command = f"""
MERGE INTO {snowflake_table} AS tgt
USING {staging_table} AS src
ON tgt.PRODUCT_NAME = src.PRODUCT_NAME
    AND tgt.COMPANY = src.COMPANY
    AND tgt.DATE_INSERTED = src.DATE_INSERTED
WHEN MATCHED THEN
    UPDATE SET
        tgt.PRODUCT_DESCRIPTION = src.PRODUCT_DESCRIPTION,
        tgt.SUGGESTED_KEYWORDS = src.SUGGESTED_KEYWORDS,
        tgt.QUERY_KEYWORDS = src.QUERY_KEYWORDS,
        tgt.BLOG_POST = src.BLOG_POST,
        tgt.SOCIAL_MEDIA_POST = src.SOCIAL_MEDIA_POST,
        tgt.DATE_INSERTED = src.DATE_INSERTED
WHEN NOT MATCHED THEN
    INSERT (PRODUCT_NAME, PRODUCT_DESCRIPTION, SUGGESTED_KEYWORDS, QUERY_KEYWORDS, BLOG_POST, SOCIAL_MEDIA_POST, DATE_INSERTED, COMPANY)
    VALUES (src.PRODUCT_NAME, src.PRODUCT_DESCRIPTION, src.SUGGESTED_KEYWORDS, src.QUERY_KEYWORDS, src.BLOG_POST, src.SOCIAL_MEDIA_POST, src.DATE_INSERTED, src.COMPANY);
"""

# Tracking command to log the merge
tracking_command = """
INSERT INTO GOOGLE_BIGQUERY.EVENT_VIEWS.RUNTIME_HST (table_name, table_update, timestamp)
VALUES (%s, %s, %s);
"""
truncate = "truncate GOOGLE_BIGQUERY.SEARCH_CONSOLE_STG.PAGE_CATEGORY_DATA_STG;"
cur = conn.cursor()

cur.execute(copy_command)
cur.execute(tracking_command, (snowflake_table, 'INSERT', datetime.now()))
cur.execute(merge_command)
cur.execute(truncate)

conn.commit()
conn.close()



as
  Product Name         Product Description  \
0        Urea   urea fertilizer for plants   
1        Urea      type of urea fertilizer   
2        Urea   urea fertilizer for plants   
3        Urea     urea nitrogen fertiliser   
4  Corn Gluten            lawn corn gluten   
5        Urea     fertilizer urea nitrogen   
6        Urea   urea fertilizer for plants   
7  Corn Gluten       corn gluten for grass   
8  Corn Gluten       corn gluten for grass   
9  Corn Gluten            lawn corn gluten   

                   Suggested Keywords                         Query Keywords  \
0   urea fertilizer price in pakistan                  urea fertilizer price   
1  effect of urea fertilizer on maize                  urea fertilizer price   
2      advantages of urea fertilizers                  urea fertilizer price   
3          what is urea in fertilizer                  urea fertilizer price   
4          gluten free corn dog bites  are kellogg's corn flakes gluten free   
5      adv